In [1]:
import theano
import numpy as np
import scipy.sparse as sp
from theano import sparse

In [2]:
sparse.all_dtypes

{'complex128',
 'complex64',
 'float32',
 'float64',
 'int16',
 'int32',
 'int64',
 'int8',
 'uint16',
 'uint32',
 'uint64',
 'uint8'}

In [3]:
x = sparse.csc_matrix(name='x', dtype='float32')
y = sparse.dense_from_sparse(x)
z = sparse.csc_from_dense(y)
print x, y, z

SparseVariable{csc,float32} DenseFromSparse{structured_grad=True}.0 SparseVariable{csc,float32}


In [5]:
x = sparse.csc_matrix(name='x', dtype='int32')
data, indices, indptr, shape = sparse.csm_properties(x)
y = sparse.CSR(data, indices, indptr, shape)
f = theano.function([x], y)
a = sp.csc_matrix(np.asarray([[0, 1, 1], [0, 0, 0], [1, 0, 0]]))
print a.toarray()
print f(a).toarray()

[[0 1 1]
 [0 0 0]
 [1 0 0]]
[[0 0 1]
 [1 0 0]
 [1 0 0]]


In [6]:
x = sparse.csc_matrix(name='x', dtype='float64')
y = sparse.structured_add(x, 2)
f = theano.function([x], y)
a = sp.csc_matrix(np.asarray([[0, 0, -1], [0, -2, 1], [3, 0, 0]], dtype='float64'))
print a
print
print a.toarray()
print
print f(a)
print
print f(a).toarray()

  (2, 0)	3.0
  (1, 1)	-2.0
  (0, 2)	-1.0
  (1, 2)	1.0

[[ 0.  0. -1.]
 [ 0. -2.  1.]
 [ 3.  0.  0.]]

  (2, 0)	5.0
  (1, 1)	0.0
  (0, 2)	1.0
  (1, 2)	3.0

[[ 0.  0.  1.]
 [ 0.  0.  3.]
 [ 5.  0.  0.]]


In [7]:
"""
a = sp.rand(10,100000,0.0001,format='csc')
b = sp.rand(100000,100000,0.0001,format='csc')
x = sparse.csc_matrix(name='x', dtype='float64')
y = sparse.csc_matrix(name='y', dtype='float64')
z = (x * y)
f = theano.function([x,y], z)

print f(a,b).sum()
# print f(a,b).toarray()
print 100000*100000*2*8/1000000000, " GB !"
"""

'\na = sp.rand(10,100000,0.0001,format=\'csc\')\nb = sp.rand(100000,100000,0.0001,format=\'csc\')\nx = sparse.csc_matrix(name=\'x\', dtype=\'float64\')\ny = sparse.csc_matrix(name=\'y\', dtype=\'float64\')\nz = (x * y)\nf = theano.function([x,y], z)\n\nprint f(a,b).sum()\n# print f(a,b).toarray()\nprint 100000*100000*2*8/1000000000, " GB !"\n'

In [8]:
N = int(2**26)
density = 1./N/2
mini = 50
import theano.tensor as T

w = T.matrix(name = "w", dtype = "float64")
x = sparse.csc_matrix(name="x", dtype="float64")
zr = T.maximum(sparse.structured_dot(x,w),0)
zt = T.tanh(sparse.structured_dot(x,w))
gr = theano.function([x,w],zr)
gt = theano.function([x,w],zt)

%time ww = np.random.rand(N,1)
xx = sp.rand(mini, N, density, format='csc')

%time x = gr(xx,ww)
%time y = gt(xx,ww)

%time x.sum()
%time y.sum()

Wall time: 4.43 s
Wall time: 194 ms
Wall time: 286 ms
Wall time: 0 ns
Wall time: 0 ns


5.5820384842429265

In [9]:
N = 2**26
features = 20
import theano.tensor as T
import random

w = T.matrix(name = "w", dtype = "float64")
x = sparse.csr_matrix(name="x", dtype="float64")
zr = T.maximum(sparse.structured_dot(x,w),0)
zt = T.tanh(sparse.structured_dot(x,w))
gr = theano.function([x,w],zr)
gt = theano.function([x,w],zt)

%time ww = np.random.rand(N,1)
xx = sp.lil_matrix((1,N))
for i in range(features):
    xx[0,random.randint(0,N)] = 1

%time gr(xx,ww).sum()



Wall time: 5.03 s
Wall time: 2 ms


9.670729639370478